In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import re
from PIL import Image
import shutil
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2 as cv

In [7]:
## vgg16 pretrained weights 다운로드
weight_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                      'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [8]:
IMG_SIZE = 224
def create_model():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    conv1_1 = Conv2D(64, 3, 1, 'SAME', activation='relu')(inputs)
    conv1_2 = Conv2D(64, 3, 1, 'SAME', activation='relu')(conv1_1)
    pool1_3 = MaxPooling2D()(conv1_2)
    
    conv2_1 = Conv2D(128, 3, 1, 'SAME', activation='relu')(pool1_3)
    conv2_2 = Conv2D(128, 3, 1, 'SAME', activation='relu')(conv2_1)
    pool2_3 = MaxPooling2D()(conv2_2)
    
    conv3_1 = Conv2D(256, 3, 1, 'SAME', activation='relu')(pool2_3)
    conv3_2 = Conv2D(256, 3, 1, 'SAME', activation='relu')(conv3_1)
    conv3_3 = Conv2D(256, 3, 1, 'SAME', activation='relu')(conv3_2)
    pool3_4 = MaxPooling2D()(conv3_3)
    
    conv4_1 = Conv2D(512, 3, 1, 'SAME', activation='relu')(pool3_4)
    conv4_2 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv4_1)
    conv4_3 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv4_2)
    pool4_4 = MaxPooling2D()(conv4_3)
    
    conv5_1 = Conv2D(512, 3, 1, 'SAME', activation='relu')(pool4_4)
    conv5_2 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv5_1)
    conv5_3 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv5_2)
    pool5_4 = MaxPooling2D()(conv5_3)
    
    ## loading vgg16 pretrained weights
    vgg = keras.Model(inputs, pool5_4)
    vgg.load_weights(weight_path)
    
    upconv6 = Conv2DTranspose(512, 5, 2, 'SAME', activation='relu')(pool5_4)
    concat6 = Concatenate()([conv5_3, upconv6])
    conv6 = Conv2D(512, 3, 1, 'SAME', activation='relu')(concat6)
                              
    upconv7 = Conv2DTranspose(512, 5, 2, 'SAME', activation='relu')(conv6)
    concat7 = Concatenate()([conv4_3, upconv7])
    conv7 = Conv2D(512, 3, 1, 'SAME', activation='relu')(concat7)
    
    upconv8 = Conv2DTranspose(256, 5, 2, 'SAME', activation='relu')(conv7)
    concat8 = Concatenate()([conv3_3, upconv8])
    conv8 = Conv2D(256, 3, 1, 'SAME', activation='relu')(concat8)
    
    upconv9 = Conv2DTranspose(128, 5, 2, 'SAME', activation='relu')(conv8)
    concat9 = Concatenate()([conv2_2, upconv9])
    conv9 = Conv2D(128, 3, 1, 'SAME', activation='relu')(concat9)
    
    upconv10 = Conv2DTranspose(64, 5, 2, 'SAME', activation='relu')(conv9)
    concat10 = Concatenate()([conv1_2, upconv10])
    conv10 = Conv2D(64, 3, 1, 'SAME', activation='relu')(concat10)
    
    conv11 = Conv2D(64, 3, 1, 'SAME', activation='relu')(conv10)
    
    conv12 = Conv2D(2, 1, 1, 'SAME', activation='softmax')(conv11)
    
    return keras.Model(inputs=inputs, outputs=conv12) 

model = create_model()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 224, 224, 64) 36928       conv2d_13[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 112, 112, 64) 0           conv2d_14[0][0]                  
____________________________________________________________________________________________

In [9]:
weight_path = 'G:\\My Drive\Checkpoints for Background Remover\\saved-unet_model-02-0.09.hdf5'
model.load_weights(weight_path)

path = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128'
#path = 'C:\\Users\\user\\Documents\\Namin\\sample_input'
imagePaths = [os.path.join(path,file_name) for file_name in os.listdir(path)]
outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_mask_train'
#outpath = 'C:\\Users\\user\\Documents\\Namin\\sample_mask'

# test 이미지
# camera_list = ['4','7','10'] #4, 정면은 7
# light_list = ['1'] #2
# accessory_list =['1'] #1
# expression_list = ['1','2','3'] #320

camera_list = list(map(int, [4,5,6,7,8,9,10,14,15,16,17,18,19,20])) #4, 정면은 7
light_list = list(map(int, [1,2,3,4,5,6, # 검은거(L7) 제거
    8,9,10,11,12,13,14,15, 16,17,18,
    19,20,21,22,23,24,25,26,27, 28,29,30])) #2
accessory_list = list(map(int, [1,2,3,5])) #1
expression_list = list(map(int, list(range(1, 4)))) #320

In [10]:
for imagePath in imagePaths: # 모든 사람들 폴더에 대하여
    
    folder_name = os.path.basename(imagePath) # 사람 숫자
    os.makedirs(outpath+'\\'+folder_name, exist_ok=True) # 사람 숫자 폴더 만들기
    person_outpath = outpath + '\\' + folder_name # 만들어진 사람 숫자 폴더
    filePaths = [os.path.join(imagePath, file_name) for file_name in os.listdir(imagePath)] # 한 사람 폴더 안에 들어있는 파일들

    if os.listdir(person_outpath): # 만들어진게 있으면
        print(f'Already done with {person_outpath}')
        pass
    else:
        print(imagePath)
        for filePath in filePaths: # 모든 파일들에 대하여
            
            # 파일들 중에서 관심있는 parameter들에 대해서
            file_name  = os.path.basename(filePath)
            
            light_split = file_name.split('L')
            exp_split = light_split[-1].split('E')
            ang_split = file_name.split('C')
            dot_split = ang_split[-1].split('.')
            # print(light_split[0][-1])
            # print(exp_split[0])
            # print(ang_split[0][-1])
            # print(dot_split[0])

            if (int(light_split[0][-1]) in accessory_list) and (int(exp_split[0]) in light_list) and (int(ang_split[0][-1]) in expression_list) and (int(dot_split[0]) in camera_list):
                image = Image.open(filePath)
                #image = cv.imread(file_name, cv.IMREAD_GRAYSCALE)
                #image = np.array(image)
                #print(image.shape)
                #image = np.reshape(image.shape[0]*image.shape[1], -1)
                #n, bins, patches = plt.hist(x=image, bins='auto')
                #plt.show()
                image = image.resize((224, 224))
                image = np.array(image)
                image = image/255. 
                image = np.reshape(image, (1, 224, 224, 3))

                prediction = model.predict(image)
                pred = np.zeros_like(prediction)
                ## 0.5이상은 1로 나머지는 0으로 변환
                thr = 0.5
                pred[prediction>=thr] = 255 # 얼굴
                pred[prediction<thr] = 0 # 배경(검은색)
                # plt.imshow(pred[0,:,:,1])
                mask = pred[0,:,:,1]
                mask = cv.resize(mask, dsize=(128, 128), interpolation=cv.INTER_CUBIC) 

                #head_and_tail = os.path.splitext(os.path.basename(file_name)) # ('S003L1E03C16.jpg', '.jpg')
                cv.imwrite(person_outpath+'\\'+file_name, mask)  # 여기 뒤에 0으로 하면 색깔 반대로 저장됨

Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19062542
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19062811
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19062831
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19062832
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19062842
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19070121
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19070221
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19070231
Already done with C:\Users\user\Desktop\Middle_Resolution_177_unzipped_parcropped_128_mask_train\19070241
Already done with C:\Users\user\Desktop\Middle